<a href="https://colab.research.google.com/github/importYM/proposal/blob/main/NWPPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Pandas import & data load**
* pandas import
* csv file read
* data print

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from pandas import read_csv

data = pd.read_csv('/content/sample_221212_4.csv') # read csv file 
data.columns, data, data.shape[1]

# **3. Divide to data_x, data_y**

In [ ]:
data2 = data
data2.set_index('TIME', inplace=True) # Change the index to be based on the 'TIME' column

data_x = data2.iloc[:,0:]             # data_x = nursing workload proxies grade, ID, Biomarker, Hospital demographic data 
data_y = data2.iloc[:,0:2]            # data_y = nursing workload proxies grade, ID

data_x, data_y

# **3. Setting the Maximum Data Length (hospital day 30)**

In [ ]:
# data_x에 대하여

data_x_pad = pd.DataFrame(columns=data_x.columns)              # 빈 데이터프레임 생성

for id in data_x['ID'].unique():                               # 각 인덱스별로 루프를 돌면서 ID마다 행 수를 맞추기 위한 for문
    rows = data_x[data_x['ID'] == id]
    n_rows = len(rows)
    n_padding_rows = (30 - n_rows % 30) % 30                   # 30의 배수가 되도록 패딩할 행의 수 계산
    padding_rows = pd.DataFrame({'ID': [id] * n_padding_rows,
                                 'G3': [0] * n_padding_rows,
                                 'F1': [0] * n_padding_rows,
                                 'F2': [0] * n_padding_rows,
                                 'F3': [0] * n_padding_rows})  # 패딩할 행의 값은 0으로 설정
    new_rows = pd.concat([rows, padding_rows])
    new_rows = new_rows.iloc[:30]                              # 인덱스별로 최대 30개의 행만 남기고 나머지는 버림
    data_x_pad = pd.concat([data_x_pad, new_rows])

# 결과 확인
print(data_x_pad)

In [ ]:
# data_y에 대하여

data_y_pad = pd.DataFrame(columns=data_y.columns)              # 빈 데이터프레임 생성


for id in data_y['ID'].unique():                               # 각 인덱스별로 루프를 돌면서 ID마다 행 수를 맞추기 위한 for문
    rows = data_y[data_y['ID'] == id]
    n_rows = len(rows)
    n_padding_rows = (30 - n_rows % 30) % 30                   # 30의 배수가 되도록 패딩할 행의 수 계산
    padding_rows = pd.DataFrame({'ID': [id] * n_padding_rows,
                                 'G3': [0] * n_padding_rows})  # 패딩할 행의 값은 0으로 설정
    new_rows = pd.concat([rows, padding_rows])
    new_rows = new_rows.iloc[:30]                              # 인덱스별로 최대 30개의 행만 남기고 나머지는 버림
    data_y_pad = pd.concat([data_y_pad, new_rows])

# 결과 확인
print(data_y_pad)

# **4. Change Pandas DataFrame to Numpy array**

In [ ]:
x = data_x_pad.values     # Change Pandas DataFrame to Numpy array
y = data_y_pad.values     # Change Pandas DataFrame to Numpy array
print(x, y)

In [ ]:
x.shape, y.shape          # x, y shape 확인 = 2D array

# **5. Normalization feature scaling**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

x_without_col0_1 = np.delete(x, [0, 1], axis=1)                         # delete column(nursing workload proxies grade, ID)

scaler = MinMaxScaler(feature_range=(0, 1))                             # scaler setting
x_normalized = scaler.fit_transform(x_without_col0_1)                   # scaler fit

x_normalized_with_all = np.insert(x_normalized, [0], x[:, :2], axis=1)  # add column(nursing workload proxies grade, ID)

x_normalized_with_all

# **6. Add NaN row**

In [ ]:
# x_normalized_with_all 에서

x_id = x_normalized_with_all[:, 1]                                  # ID열 추출
x_other_id = np.where(x_id[:-1] != x_id[1:])[0] + 1                 # ID가 바뀌는 인덱스 확인

NaN_x_row = np.full((1, n_x_col), np.nan)                           # 2D array x에 삽입할 NaN 행 생성
X = np.insert(x_normalized_with_all, x_other_id, NaN_x_row, axis=0) # 2D 배열에 ID가 바뀌는 행마다 NaN 값을 가진 행 삽입

print(X)

In [ ]:
# y 에서

y_id = y[:, 1]                                      # ID열 추출
y_new_id = np.where(y_id[:-1] != y_id[1:])[0] + 1   # ID가 바뀌는 인덱스 확인

NaN_y_row = np.full((1, n_y_col), np.nan)           # 2D array y에 삽입할 NaN 행 생성
Y = np.insert(y, y_new_id, NaN_x_row, axis=0)       # 2D 배열에 ID가 바뀌는 행마다 NaN 값을 가진 행 삽입

print(Y)